# Machine Enigma M3

In [1]:
class Enigma:
    
    def __init__(self):
        
        #Rotors
    
        RI = [4,10,12,5,11,6,3,16,21,25,13,19,14,22,24,7,23,20,18,15,0,8,1,17,2,9]#EKMFLGDQVZNTOWYHXUSPAIBRCJ
        RII = [0,9,3,10,18,8,17,20,23,1,11,7,22,19,12,2,16,6,25,13,15,24,5,21,14,4]#AJDKSIRUXBLHWTMCQGZNPYFVOE
        RIII = [1,3,5,7,9,11,2,15,17,19,23,21,25,13,24,4,8,22,6,0,10,12,20,18,16,14]#BDFHJLCPRTXVZNYEIWGAKMUSQO
        RIV = [4,18,14,21,15,25,9,0,24,16,20,8,17,7,23,11,13,5,19,6,10,3,2,12,22,1]#ESOVPZJAYQUIRHXLNFTGKDCMWB
        RV = [21,25,1,17,6,8,19,24,20,15,18,3,13,7,11,23,0,22,12,9,16,14,5,4,2,10]#VZBRGITYUPSDNHLXAWMJQOFECK
        
        self.Rotors = [RI,RII,RIII,RIV,RV]
        
        self.changementsRotors = [17,5,22,10,0]
        
        #Reflectors
        
        RefB = [24,17,20,7,16,18,11,3,15,23,13,6,14,10,12,8,4,1,5,25,2,22,21,9,0,19]#YRUHQSLDPXNGOKMIEBFZCWVJAT
        RefC = [17,3,14,1,9,13,19,10,21,4,7,12,11,5,2,22,25,0,23,6,24,8,15,18,20,16]#DOBJNTKVEHMLFCWZAXGYIPSUQ
        
        self.Ref=[RefB,RefC]
        
        #Plugboard of 10 connexion
        
        self.Plugboard = [i for i in range(26)]
        
        #Initial Setup
        
        self.RotorSetup = [0,1,2] #Rotor Arrangements
        self.ReflectSetup = 0 #0 for RefB and 1 for RefC
        self.RotorInitPos = [0,0,0]
        #self.Plugboard
        
        #Evolution of rotors 
        self.RotorCurrentPos = [0,0,0]
        
    def set_Rotors(self):#Force Deep copy
        for i in range(3):
            self.RotorCurrentPos[i] = self.RotorInitPos[0]
     
    def reflector(self,ind):
        return self.Ref[self.ReflectSetup][ind]
    
    def plugboard(self,ind):
        return self.Plugboard[ind]
    
    def rotor(self,number_rotor,order,ind):
        return self.Rotors[number_rotor][(ind+self.RotorCurrentPos[order])%26]
    
    def rotor_inv(self,number_rotor,order,ind):
        res = 0
        for i in range(26):
            if (ind == self.Rotors[number_rotor][(i+self.RotorCurrentPos[order])%26]):
                res = i 
        return res
    
    def show(self):
        print('\033[1m' + 'Enigma Settings' + '\033[0m')
        print("Plugboard :", self.Plugboard)
        tmp =[0,0,0]
        for i in range(3):
            tmp[i] = self.RotorSetup[i]+1
        print("Rotor Setup : ",tmp)
        print("ReflectSetut :",self.ReflectSetup)
        print("RotorInitPos :",self.RotorInitPos)
    
    def setup(self,new_Plugboard,new_RotorInitPos=['A','A','A'],new_RotorSetup=[1,2,3],new_ReflectSetup=0):
        for co in new_Plugboard:
            self.Plugboard[ord(co[0])-65] = ord(co[1])-65
            self.Plugboard[ord(co[1])-65] = ord(co[0])-65
        for i in range(3):
            self.RotorInitPos[i] = ord(new_RotorInitPos[i])-65
        for i in range(3):
            self.RotorSetup[i] = new_RotorSetup[i]-1
        self.ReflectSetup = new_ReflectSetup
        
        
    def encrypt_one(self,ind):#encrypt from a number in 0,...,25
        tmp = ind
        tmp = self.plugboard(ind)
        for i in range(3):
            tmp = self.rotor(self.RotorSetup[i],i,tmp)
        tmp = self.reflector(tmp)
        for i in range(3):
            tmp = self.rotor_inv(self.RotorSetup[2-i],2-i,tmp)
        tmp = self.plugboard(tmp)
        return tmp
    
    def encrypt_list(self,list_ind):#encrypt from a list of number in 0,...,25
        res = []
        for ind in list_ind:
            res.append(self.encrypt_one(ind))
            self.RotorCurrentPos[0] = (self.RotorCurrentPos[0]+1)%26
            if (self.RotorCurrentPos[0] == self.changementsRotors[self.RotorSetup[0]]):
                self.RotorCurrentPos[1] = (self.RotorCurrentPos[1]+1)%26
            if (self.RotorCurrentPos[1] == self.changementsRotors[self.RotorSetup[1]]):
                self.RotorCurrentPos[2] = (self.RotorCurrentPos[2]+1)%26
        return res
    
    def encrypt_word(self,word):#crypte 1 word write in Uppercase
        list_ind = []
        for char in word:
            list_ind.append(ord(char)-65)
        tmp = self.encrypt_list(list_ind)
        res = ''
        for ind in tmp:
            res = res + chr(ind+65)
        return res
    
    def encrypt_sentence(self,sentence):
        self.set_Rotors()
        res = ""
        for word in sentence.split():
            res = res + " " + self.encrypt_word(word)
        return res[1:]
    
    def encrypt_file(self,pathfile_clean,pathfile_encrypt):
        with open(pathfile_clean, 'r') as myfile:
            sentence = self.encrypt_sentence(myfile.read())
        with open(pathfile_encrypt,'w') as myfile:
            myfile.write(sentence)

Create and setup our machine

In [2]:
enigma = Enigma()
enigma.show()

Enigma Settings
Plugboard : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
Rotor Setup :  [1, 2, 3]
ReflectSetut : 0
RotorInitPos : [0, 0, 0]


## 1st Test

In [3]:
sentence ="JE SUIS EN RETARD JE SUIS DESOLER"
print(sentence)
res = enigma.encrypt_sentence(sentence)
print(res)
res_inv = enigma.encrypt_sentence(res)
print(res_inv)
print("Succes :",sentence==res_inv)

JE SUIS EN RETARD JE SUIS DESOLER
GA BZSX IW QRKEIF OG BARY TICUCAL
JE SUIS EN RETARD JE SUIS DESOLER
Succes : True


Setup our machine on different configurations

In [4]:
new_plug = ['AW','HX','CO','JP','EK','FR']
new_rotorpos = ['M','R','Z']
new_rotsetup = [5,2,3]
new_refsetup = 1
enigma.setup(new_plug,new_rotorpos,new_rotsetup,new_refsetup)
enigma.show()

Enigma Settings
Plugboard : [22, 1, 14, 3, 10, 17, 6, 23, 8, 15, 4, 11, 12, 13, 2, 9, 16, 5, 18, 19, 20, 21, 0, 7, 24, 25]
Rotor Setup :  [5, 2, 3]
ReflectSetut : 1
RotorInitPos : [12, 17, 25]


## 2nd Test

In [5]:
sentence ="I REALLY MISSED TIME I SWAM EVERYDAY"
print(sentence)
res = enigma.encrypt_sentence(sentence)
print(res)
res_inv = enigma.encrypt_sentence(res)
print(res_inv)
print("Succes :",sentence==res_inv)

I REALLY MISSED TIME I SWAM EVERYDAY
Y SXCWSP ZJRDAS GKNW C CEGS CZCEUISZ
I REALLY MISSED TIME I SWAM EVERYDAY
Succes : True


## Main

In [6]:
enigma.encrypt_file('message_clean.txt','message_encrypt.txt')